In [3]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical
f = open("big.txt")



LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Vineeth/nltk_data'
    - 'f:\\pro\\env\\nltk_data'
    - 'f:\\pro\\env\\share\\nltk_data'
    - 'f:\\pro\\env\\lib\\nltk_data'
    - 'C:\\Users\\Vineeth\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vineeth\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [9]:
f = open("big.txt")
data = f.read()
len(data)

3603366

In [39]:
cleaned = re.sub(r'\W+', ' ',data[:100000]).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [43]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])


(18683, 3)

In [41]:
train_targets[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3, 3)              9417      
_________________________________________________________________
lstm_12 (LSTM)               (None, 3, 50)             10800     
_________________________________________________________________
lstm_13 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_12 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_13 (Dense)             (None, 3139)              160089    
Total params: 203,056
Trainable params: 203,056
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
584/584 [==============================] - 26s 22ms/step - loss: 6.9753 - accuracy: 0.0440
Epoch 

584/584 [==============================] - 13s 23ms/step - loss: 3.1092 - accuracy: 0.2781
Epoch 68/500
584/584 [==============================] - 13s 22ms/step - loss: 3.0980 - accuracy: 0.2763
Epoch 69/500
584/584 [==============================] - 13s 22ms/step - loss: 3.1023 - accuracy: 0.2775
Epoch 70/500
584/584 [==============================] - 13s 23ms/step - loss: 3.0875 - accuracy: 0.2765
Epoch 71/500
584/584 [==============================] - ETA: 0s - loss: 3.0762 - accuracy: 0.27 - 13s 23ms/step - loss: 3.0763 - accuracy: 0.2792
Epoch 72/500
584/584 [==============================] - 14s 23ms/step - loss: 3.0682 - accuracy: 0.28490s - loss: 3.0662 
Epoch 73/500
584/584 [==============================] - 13s 22ms/step - loss: 3.0020 - accuracy: 0.2960
Epoch 74/500
584/584 [==============================] - 14s 24ms/step - loss: 2.9938 - accuracy: 0.2952
Epoch 75/500
584/584 [==============================] - 13s 23ms/step - loss: 3.0025 - accuracy: 0.2941
Epoch 76/500
584/

584/584 [==============================] - 16s 27ms/step - loss: 2.4359 - accuracy: 0.3998
Epoch 142/500
573/584 [============================>.] - ETA: 0s - loss: 2.4323 - accuracy: 0.3999

In [22]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

no time to
[77, 142, 4] [[ 77 142   4]]
Next word suggestion: our
Next word suggestion: an
Next word suggestion: nothing
